<a href="https://colab.research.google.com/github/djgarayb/RNA-Seq_introduction/blob/master/S4_Exploratory_Data__Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount Google (G) drive

In [0]:
from google.colab import drive
drive.mount('/content/drive')

# Install packages

In [0]:
!Rscript -e "options(Ncpus = 2)" -e "install.packages('devtools')" -e "install.packages('BiocManager')" -e "BiocManager::install(c('edgeR','limma','tximport','biomaRt', 'dplyr', 'tidyverse','ensembldb','EnsDb.Hsapiens.v86','rhdf5','genefilter'))" -e "devtools::install_github('pachterlab/sleuth')"

In [0]:
from rpy2.robjects.packages import importr
utils = importr('utils')
%load_ext rpy2.ipython

# Set working directory (WD)
### Copy and paste the path to your kallisto_results file on G drive

In [0]:
%%R
setwd("/content/drive/My Drive/kalisto_results")

In [0]:
%ls

# Load packages

In [0]:
%%R
# Load all the R libraries we will be using in the notebook
library(tximport)
library(biomaRt)
library(Biobase)
library(ggplot2)
library(dplyr)
library(tidyverse) 
library(Biostrings)
library(ensembldb)
library(EnsDb.Hsapiens.v86) 
library(rhdf5)
library(genefilter)
library(RColorBrewer) 
library(reshape2)
library(edgeR)
library(matrixStats) 
library(sleuth) 

# Load the identifier conversion data

In [0]:
%%R
listTables(EnsDb.Hsapiens.v86)
listColumns(EnsDb.Hsapiens.v86, "tx")
Tx <- transcripts(EnsDb.Hsapiens.v86, columns=c(listColumns(EnsDb.Hsapiens.v86,"tx"), "gene_name"))
Tx <- as_tibble(Tx)
Tx <- dplyr::rename(Tx, target_id = tx_id)
Tx <- dplyr::select(Tx, target_id, gene_name)
print(dim(Tx))
head(Tx)

# Create and load metadata file


*   Create your metadata file in excel using sample, folder, group as mandatory columns 
*   You can add more columns
*   Save it as .csv
*   Load it on your notebook

In [0]:
%%R
metadata <- read.csv ('Metadata_susp.csv', header=TRUE)
#metadata <- as.data.frame(metadata)
metadata

In [0]:
%%R
path <- file.path(metadata$folder, "abundance.h5")
all(file.exists(path)) 

In [0]:
%%R
Txi_gene <- tximport(path, 
                     type = "kallisto", 
                     tx2gene = Tx, 
                     txOut = FALSE, #How does the result change if this =FALSE vs =TRUE?
                     countsFromAbundance = "lengthScaledTPM",
                     ignoreTxVersion=TRUE)

colSums(Txi_gene$counts)

# Examine your data up to this point

In [0]:
%%R
myTPM <- Txi_gene$abundance
myCounts <- Txi_gene$counts
colnames(myCounts) <- metadata$folder
colnames(myTPM) <- metadata$folder
print(colSums(myTPM))
colSums(myCounts)

# Generate summary stats for your data ----
- 1st, calculate summary stats for each transcript or gene, and add these to your data matrix
- then use the base R function 'transform' to modify the data matrix (equivalent of Excel's '=')

In [0]:
%%R
myTPM.stats <- transform(myTPM, 
                         SD=rowSds(myTPM), 
                         AVG=rowMeans(myTPM),
                         MED=rowMedians(myTPM)
                         )

myCounts.stats <- transform(myCounts, 
                            SD=rowSds(myCounts), 
                            AVG=rowMeans(myCounts), 
                            MED=rowMedians(myCounts)
                            )
head(myTPM.stats)

In [0]:
%%R
 ggplot(myCounts.stats, aes(x=MED, y=SD)) +
  geom_point(shape=1, size=4)


In [0]:
%%R
 ggplot(myCounts.stats, aes(x=MED, y=SD)) +
  geom_point(shape=1, size=4)+
  scale_x_continuous(trans='log2')+
  scale_y_continuous(trans='log2')

# Make a DGElist from your counts ----

In [0]:
%%R
myDGEList <- DGEList(Txi_gene$counts)
# take a look at the DGEList object 
myDGEList
#DEGList objects are a good R data file to consider saving to you working directory
save(myDGEList, file = "myDGEList")
#Saved DGEList objects can be easily shared and loaded into an R environment
load(file = "myDGEList")

# Use the 'cpm' function from EdgeR to get counts per million

In [0]:
%%R
cpm <- cpm(myDGEList) 
print(colSums(cpm))
log2.cpm <- cpm(myDGEList, log=TRUE)

# Take a look at the distribution of the Log2 CPM
nsamples <- ncol(log2.cpm)
myColors <- brewer.pal(nsamples, "Paired")

# 'Coerce' your data matrix to a dataframe so that you can use tidyverse tools on it

In [0]:
%%R
log2.cpm.df <- as.tibble(log2.cpm)
log2.cpm.df
# add your sample names to this dataframe (we lost these when we read our data in with tximport)
colnames(log2.cpm.df) <- metadata$folder
# use the reshape2 package to 'melt' your dataframe (from wide to tall)
log2.cpm.df.melt <- melt(log2.cpm.df)
Log2.cpm.df.melt <- as.tibble(log2.cpm.df.melt)
Log2.cpm.df.melt

In [0]:
%%R
ggplot(Log2.cpm.df.melt, aes(x=variable, y=value, fill=variable)) +
geom_violin(trim = FALSE, show.legend = TRUE) +
stat_summary(fun.y = "median", geom = "point", shape = 95, size = 10, color = "black") +
theme_bw()

# Filter your data ----

In [0]:
%%R
#first, take a look at how many genes or transcripts have no read counts at all
table(rowSums(myDGEList$counts==0)==16)
DGEList <- myDGEList


# now set some cut-off to get rid of genes/transcripts with low counts
keepers <- rowSums(cpm>1)>=6
DGEList.filtered <- DGEList[keepers,]
dim(DGEList.filtered)

log2.cpm.filtered <- cpm(DGEList.filtered, log=TRUE)
log2.cpm.filtered.df <- as.tibble(log2.cpm.filtered) 
colnames(log2.cpm.filtered.df) <- metadata$folder
log2.cpm.filtered.df.melt <- melt(log2.cpm.filtered.df)
log2.cpm.filtered.df.melt <- as.tibble(log2.cpm.filtered.df.melt)

In [0]:
%%R
ggplot(log2.cpm.filtered.df.melt, aes(x=variable, y=value, fill=variable)) +
geom_violin(trim = TRUE, show.legend = TRUE) +
stat_summary(fun.y = "median", geom = "point", shape = 95, size = 10, color = "black") +
theme_bw()

# Normalize your data ----

In [0]:
%%R
DGEList.filtered.norm <- calcNormFactors(DGEList.filtered, method = "TMM")
# take a look at this new DGEList object...how has it changed?

# use the 'cpm' function from EdgeR to get counts per million from your normalized data
log2.cpm.filtered.norm <- cpm(DGEList.filtered.norm, log=TRUE)

log2.cpm.filtered.norm.df <- as.tibble(log2.cpm.filtered.norm) 
colnames(log2.cpm.filtered.norm.df) <- metadata$folder
log2.cpm.filtered.norm.df.melt <- melt(log2.cpm.filtered.norm.df)
log2.cpm.filtered.norm.df.melt <- as.tibble(log2.cpm.filtered.norm.df.melt)


In [0]:
%%R
ggplot(log2.cpm.filtered.norm.df.melt, aes(x=variable, y=value, fill=variable)) +
geom_violin(trim = TRUE, show.legend = TRUE) +
stat_summary(fun.y = "median", geom = "point", shape = 95, size = 10, color = "black") +
theme_bw()




# Hierarchical clustering ---------------

In [0]:
%%R
#hierarchical clustering can only work on a data matrix, not a data frame
#try using filtered and unfiltered data...how does this change the results
distance <- dist(t(log2.cpm.filtered.norm), method="maximum") #other dist methods are "maximum", "manhattan", "canberra", "binary" or "minkowski"
clusters <- hclust(distance, method = "complete") #other methods are ward.D, ward.D2, single, complete, average
plot(clusters, labels=metadata$folder)


# Pricipal component analysis (PCA) -------------

In [0]:
%%R
pca.res <- prcomp(t(log2.cpm.filtered.norm), scale.=F, retx=T)
#look at pca.res in environment
ls(pca.res)
summary(pca.res) # Prints variance summary for all principal components.
pca.res$rotation #$rotation shows you how much each gene influenced each PC (called 'scores')
pca.res$x #$x shows you how much each sample influenced each PC (called 'loadings')
#note that these loadings have a magnitude and a direction (this is the basis for making a PCA plot)
pc.var<-pca.res$sdev^2 #sdev^2 gives you the eigenvalues
pc.per<-round(pc.var/sum(pc.var)*100, 1)
pc.per

In [0]:
%%R
pca.res.df <- as.tibble(pca.res$x)
ggplot(pca.res.df, aes(x=PC1, y=PC2, color=metadata$group)) +
geom_point(size=5) +
theme(legend.position="right") 

In [0]:
%%R
#  Make IQR_Median Plots
counts_filtered <- DGEList.filtered.norm$counts
log_counts<-log(counts_filtered+1)
CPM<-cpm(DGEList.filtered.norm)

IQR<-apply(CPM, 2, IQR)
Median<-apply(CPM, 2, median)
diff1<-mean(Median)-min(Median)
diff2<-max(Median)-mean(Median)
diff3<-mean(IQR)-min(IQR)
diff4<-max(IQR)-mean(IQR)

#  These plot settings work for this data and may need to be adjusted for your own data.  
Xlim=c(mean(Median)-2*diff1,mean(Median)+2*diff2)
Ylim=c(mean(IQR)-2*diff3,mean(IQR)+2*diff4)


plot(Median, IQR, main="IQR vs. Median TMM normaliztion", type="n", xlim=Xlim,ylim=Ylim)
text(Median, IQR, labels=names(IQR))

#  Make boxes for StDev.
Median_mean<-mean(Median)  
c_sd1_mean<-sd(Median)
c_sd2_mean<-2*sd(Median)
c_sd3_mean<-3*sd(Median)
IQR_mean<-mean(IQR)
c_sd1_IQR<-sd(IQR)
c_sd2_IQR<-2*sd(IQR)
c_sd3_IQR<-3*sd(IQR)

x0_c<-Median_mean-c_sd1_mean
y0_c<-IQR_mean-c_sd1_IQR
x1_c<-Median_mean+c_sd1_mean
y1_c<-IQR_mean+c_sd1_IQR

x0_c.2<-Median_mean-c_sd2_mean
y0_c.2<-IQR_mean-c_sd2_IQR
x1_c.2<-Median_mean+c_sd2_mean
y1_c.2<-IQR_mean+c_sd2_IQR

x0_c.3<-Median_mean-c_sd3_mean
y0_c.3<-IQR_mean-c_sd3_IQR
x1_c.3<-Median_mean+c_sd3_mean
y1_c.3<-IQR_mean+c_sd3_IQR

segments(x0_c,y0_c, x1=x1_c, y1=y0_c, col="blue")
segments(x0_c,y0_c, x1=x0_c, y1=y1_c, col="blue")
segments(x1_c,y0_c, x1=x1_c, y1=y1_c, col="blue")
segments(x0_c,y1_c, x1=x1_c, y1=y1_c, col="blue")

segments(x0_c.2,y0_c.2, x1=x1_c.2, y1=y0_c.2, col="red")
segments(x0_c.2,y0_c.2, x1=x0_c.2, y1=y1_c.2, col="red")
segments(x1_c.2,y0_c.2, x1=x1_c.2, y1=y1_c.2, col="red")
segments(x0_c.2,y1_c.2, x1=x1_c.2, y1=y1_c.2, col="red")

#  This portion is out of range so I removed it.  
segments(x0_c.3,y0_c.3, x1=x1_c.3, y1=y0_c.3, col="green")
segments(x0_c.3,y0_c.3, x1=x0_c.3, y1=y1_c.3, col="green")
segments(x1_c.3,y0_c.3, x1=x1_c.3, y1=y1_c.3, col="green")
segments(x0_c.3,y1_c.3, x1=x1_c.3, y1=y1_c.3, col="green")


# Exclude samples if necessary

In [0]:
metadata0 <- dplyr::filter(metadata, sample != 'Sample_905_Mtb')

# Sleuth

In [0]:
%ls

# Get TTG file

In [0]:
%%R
mart <- biomaRt::useMart(biomart = "ENSEMBL_MART_ENSEMBL", dataset = "hsapiens_gene_ensembl")
ttg <- biomaRt::getBM(
  attributes = c("ensembl_transcript_id_version", "transcript_version",
                 "ensembl_gene_id", "external_gene_name", "description",
                 "transcript_biotype"),  mart = mart)
ttg <- dplyr::rename(ttg, target_id = ensembl_transcript_id_version,
                     ens_gene = ensembl_gene_id, ext_gene = external_gene_name)
ttg <- dplyr::select(ttg, c('target_id', 'ens_gene', 'ext_gene'))
head(ttg)

## Create a Sleuth Object 

In [0]:
%%R
metadata <- dplyr::mutate(metadata,
                          path = file.path(metadata$folder, "abundance.h5"))

In [0]:
%%R
head(metadata)

In [0]:
%%R
so_GS <- sleuth_prep(metadata, target_mapping = ttg,gene_mode=TRUE,
                   aggregation_column = 'ext_gene', extra_bootstrap_summary = TRUE,read_bootstrap_tpm = TRUE)

In [0]:
%%R
plot_pca(so_GS, color_by = 'group', units='scaled_reads_per_base')
new_position_theme <- theme(legend.position = c(0.10, 0.90))
plot_pca(so_GS, color_by = 'group', text_labels = TRUE,units="scaled_reads_per_base") +
  new_position_theme

In [0]:
%%R
plot_loadings(so_GS, pc_input = 1, units='scaled_reads_per_base')

In [0]:
%%R
plot_bootstrap(so_GS, "MTRNR2L12", units = "scaled_reads_per_base", color_by = "group")

In [0]:
%%
saveRDS(So_GS)